# -------------------------------
# MI Logit Lens Notebook
# -------------------------------

In [1]:
from pathlib import Path
from datasets import load_dataset, DownloadMode
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from quant_configs.bnb_configs import load_bnb_in_8bit, load_bnb_in_4bit
from mi_utils.util.logit_lens_utils.logit_lens_wrapper import LogitLensWrapper

from mi_utils.logit_lens.logit_lens_degradation_analysis import (
    analyze_UNSAFE_degradation,
    analyze_SAFE_degradation,
)

from mi_utils.logit_lens.logit_lens_analysis import run_logit_lens

from mi_utils.logit_lens.metric_utils.logit_lens_helpers import (
    save_results_to_csv,
    get_activation_tensor,
    extract_activations,
    load_results_from_pt
)

from mi_utils.logit_lens.metric_utils.interp_degradation_scores import (
    degradation_diff_score,
    interpretability_diff_score,
    degradation_score
)

from mi_utils.logit_lens.plotting_utils.lens_plotting import (
    plot_layer_metric_two_dfs,
    plot_layer_deviation,
    qq_plot_probs
)

from mi_utils.logit_lens.metric_utils.carry_over_safe_metrics import get_carry_over_safe_with_embedding

# -------------------------------
# Models
# -------------------------------

In [2]:
from enum import Enum

class Models(Enum):
    GPT2 = "Models/GPT2"
    LAIN8B = "Models/LLaMA3Instruct"
    HF100B = "Models/HF1BitLLM100Btokens"
    HF10BL = "Models/HF1BitLLMLinear10B"
    HF10BS = "Models/HF1BitLLMSigmoid10B"
    OL1B = "Models/OLMo1B"
    OL7B = "Models/OLMo1B"
    DH3B = "DHLLaMA3B"
    DH8B = "DHLLaMA8B"

class Names(Enum):
    GPT2 = "GPT2"
    LAIN8B = "Meta-Llama-3-8B-Instruct"
    HF100B = "Models/Llama3-8B-1.58-100B-tokens"
    HF10BL = "Llama3-8B-1.58-Linear-10B-tokens"
    HF10BS = "Llama3-8B-1.58-Sigmoid-k100-10B-tokens"
    OL1B = "OLMo-1B-hf"
    OL7B = "OLMo-7B-hf"
    DH3B = "DeepHermes-3-Llama-3-3B-Preview"
    DH8B = "DeepHermes-3-Llama-3-8B-Preview"

In [3]:
def load_model_and_tok(
        model_name:str,
        output_hidden_states:bool=True,
        low_cpu_mem_usage:bool=True,
        local_files_only:bool=True,
        device_map:str="cuda",
        dtype=torch.bfloat16
) -> tuple[AutoModelForCausalLM, AutoTokenizer]:
    
    tok = AutoTokenizer.from_pretrained(model_name)

    model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #output_hidden_states=output_hidden_states,
    low_cpu_mem_usage=low_cpu_mem_usage,
    local_files_only=local_files_only,
    device_map=device_map,
    torch_dtype=dtype
    )

    return model, tok

In [ ]:
def save_fp_acts_to_pt(fp_acts, save_name:str) -> None:
    fp_acts_to_save = {}
    for lname, act in fp_acts.items():
        fp_acts_to_save[lname] = {
            "hidden": act["hidden"].detach().cpu(),
            "mask": act["mask"].detach().cpu()
        }

    torch.save(fp_acts_to_save, f"logs/cka_svcca_acts/{save_name}.pt")

def load_fp_acts(fp_acts_name:str):
    fp_acts = torch.load(f"logs/cka_svcca_acts/{fp_acts_name}.pt")
    return fp_acts

# -------------------------------
# Parameters & Datasets
# -------------------------------

In [ ]:
LL_DIR = "logs/logit_lens_logs"

BATCH_DIR = f"{LL_DIR}/batch_analysis"
INTERP_DIR = f"{LL_DIR}/interp_analysis"

NQ_DIR = "natural_questions"
GS_DIR = "gsm8k"

In [ ]:
EPS = 1e-12 
TOPK = 5

In [ ]:
GPT_L = ["final_layernorm", "lm_head"]
LLAMA_L = ["norm", "lm_head"]

In [ ]:
texts = ["Hello world", "This is a test"]

In [ ]:
filepath = r'D:\LogitLensData\nq'

destination_path = str(Path(filepath))
nq_dataset = load_dataset(
    'sentence-transformers/natural-questions',
    split={
        'train': 'train[:10]'
    },
    cache_dir=destination_path,
    download_mode=DownloadMode.REUSE_DATASET_IF_EXISTS,
    keep_in_memory=True
)

In [ ]:
nq_queries = list(nq_dataset['train']['query'])
nq_answers = list(nq_dataset['train']['answer'])

In [ ]:
nq_queries = nq_dataset['train']['query']
nq_answers = nq_dataset['train']['answer']

In [ ]:
filepath = r'D:\LogitLensData\gsm8k'

destination_path = str(Path(filepath))
gsm8k_dataset = load_dataset(
    'gsm8k', 'main',
    split={
        'train': 'train[:20]'
    },
    cache_dir=destination_path,
    download_mode=DownloadMode.REUSE_DATASET_IF_EXISTS,
    keep_in_memory=True
)

In [ ]:
gsm8k_questions = gsm8k_dataset['train']['question']
gsm8k_answers = gsm8k_dataset['train']['answer']

In [ ]:
DATA = nq_answers

# -------------------------------
# Extract and save FP Activations for CKA/SVCCA
# -------------------------------

In [ ]:
llama_fp, llama_tok = load_model_and_tok(Models.LAIN8B.value, device_map="cpu", dtype=torch.float32)

In [ ]:
llama_fp_wrapper = LogitLensWrapper(
    model=llama_fp,
    tokenizer=llama_tok,
    block_step=1,
    include_input=True,
    force_include_output=True,
    include_subblocks=True,
    decoder_layer_names=LLAMA_L,
    device="cpu"
)

In [ ]:
llama_fp_acts = extract_activations(wrapper=llama_fp_wrapper, prompts=nq_answers)

In [ ]:
save_fp_acts_to_pt(llama_fp_acts, "llama_fp_acts_nq_answers")

# -------------------------------
# 2. HF1BitLLM/Llama3-8B-1.58-100B-tokens
# -------------------------------

In [ ]:
#hf100b_m, hf100b_tok = load_model_and_tok(Models.HF100B.value, dtype=torch.bfloat16)
hf100b_m, hf100b_tok = load_model_and_tok(Models.HF100B.value, device_map="cpu", dtype=torch.float32)

In [ ]:
hf100b_wrapper = LogitLensWrapper(
    model=hf100b_m,
    tokenizer=hf100b_tok,
    block_step=1,
    include_input=True,
    force_include_output=True,
    include_subblocks=True,
    decoder_layer_names=LLAMA_L, 
    device="cpu"
)

In [ ]:
hf100b_unsafe_results = analyze_UNSAFE_degradation(hf100b_wrapper, texts, top_k=5, decoder=None)

In [ ]:
save_results_to_csv(hf100b_unsafe_results, filename=f"{LL_DIR}hf100b_unsafe_results.csv")

In [ ]:
hf100b_section_scores, hf100b_overall_score = degradation_score(hf100b_unsafe_results)

print("Section scores:", hf100b_section_scores)
print("Overall interpretability score:", hf100b_overall_score)

In [ ]:
hf_acts = extract_activations(wrapper=hf100b_wrapper, prompts=texts)

In [ ]:
save_fp_acts_to_pt(hf_acts, "hf_acts")

In [ ]:
llama_fp_acts = load_fp_acts("llama_fp_acts_nq_answers")

In [ ]:
hf_acts = extract_activations(wrapper=hf100b_wrapper, prompts=DATA)

In [ ]:
save_fp_acts_to_pt(hf_acts, "hf100b_acts_nq_answers")

In [ ]:
hf_acts = load_fp_acts("hf100b_acts_nq_answers")

In [ ]:
run_logit_lens(
    wrapper=hf100b_wrapper,
    prompts=DATA,
    model_name="llama_hf100b",
    dataset_name="nq_answers",
    topk=TOPK,
    eps=EPS,
    skip_input_layer= False, # True to skip embedding
    include_final_norm=True,
    save_layer_probs=False
)

In [5]:
import torch

data = torch.load(
    "logs/logit_lens_logs/logit_lens_analysis/nq_answers_llama_hf100b.pt",
    weights_only=False  # allow full unpickling
)


In [6]:
import pandas as pd
hf_df = pd.DataFrame(data)

In [7]:
get_carry_over_safe_with_embedding(hf_df, topk=5, prefix='layers.')

/media/am/AM/LogitDiffLens/mi_utils/logit_lens/metric_utils/carry_over_safe_metrics.py:155: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return layers_tensor, torch.tensor(input_ids, dtype=torch.long), torch.tensor(target_ids, dtype=torch.long)


{'per_prompt': {0: {'acc_top1_scalar': 0.16129031777381897,
   'acc_topk_scalar': 0.35483869910240173,
   'persistency_top1_scalar': 0.5078043341636658,
   'persistency_topk_scalar': 0.061394378542900085,
   'consistency_top1_scalar': 0.09216590225696564,
   'consistency_topk_scalar': 0.2617318332195282,
   'earliness_top1_scalar': 0.0208116564899683,
   'earliness_topk_scalar': 0.08428719639778137},
  1: {'acc_top1_scalar': 0.09677419066429138,
   'acc_topk_scalar': 0.22580644488334656,
   'persistency_top1_scalar': 0.4682621657848358,
   'persistency_topk_scalar': 0.05515088513493538,
   'consistency_top1_scalar': 0.07526881992816925,
   'consistency_topk_scalar': 0.19204531610012054,
   'earliness_top1_scalar': 0.002081165323033929,
   'earliness_topk_scalar': 0.04058272764086723},
  2: {'acc_top1_scalar': 0.09677419066429138,
   'acc_topk_scalar': 0.25806450843811035,
   'persistency_top1_scalar': 0.4526534974575043,
   'persistency_topk_scalar': 0.07492194324731827,
   'consistenc

In [ ]:
hf_df.columns

In [ ]:
df["layer_name"].unique()

In [ ]:
hf_df.isna().sum()

In [ ]:
pos_inf_counts = df.apply(lambda col: (col == np.inf).sum())
neg_inf_counts = df.apply(lambda col: (col == -np.inf).sum())
print(f"{neg_inf_counts}\n{pos_inf_counts}")

In [ ]:
#model.save_pretrained("Models/", safe_serialization=True)
#tokenizer.save_pretrained("Models/")